# Image Recoloring with conditional GANs

Project developed by <b>Alejandro Cano Caldero</b> and <b>Jesús Moncada Ramírez</b> for the subject <i>Neural Networks and Deep Learning</i>, University of Padova 2022-2023.

In [2]:
# All our imports will be here
from PIL import Image

from torch.utils.data import Dataset, DataLoader
import numpy as np

## 1. Dataset

In this section we will create a <i>Torch Dataset</i> class for all the images we are going to manage during this project. In a first version of this project we will use a subset of <i>ImageNet</i> called [Imagenette](https://github.com/fastai/imagenette.git) (the full size version).

In [ ]:
# Jesús: I think we will have to use this structure
class ImageDataset(Dataset):

    def __init__(self, dataset_folder, dataset_type, transform=None):
        self.x = 
        self.transform = transform
    
    def __len__(self):
        return min(32000, len(self.x))
        #return len(self.x)

    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx, 0, 0].astype(float)
        if self.transform:
            x = self.transform(x)
        return x, y

In [3]:
class GrayScaleImage():
    
    def __init__(self, images):
        self.images = images
    
    def dataset2gray(self):
        return map(lambda img: img.convert('L'), self.images)
        

In [7]:
img_dir = "C:/Alejandro/Images/meme2.jpg"
img_rgb = Image.open(img_dir)
img_rgb.show()

In [8]:
images = [img_rgb]
test = GrayScaleImage(images)
img_gray = test.dataset2gray()

for i in img_gray:
    i.show()